# **Estimating Parameters of a Gamma Distribution**

**Import Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma
from scipy.optimize import minimize

# Data Generation

Simulate a sample data of size n from a Gamma(α, β) distribution using known
parameters

Use at least five different size scales for n and use 10 different settings of α and β

5 Different Sample Sizes

In [ ]:
SampleSize = [10, 100, 1000, 10000, 100000]

Randomize 10 different Alpha and Beta pairs

In [ ]:
AlphaBetaPairs = []

for i in range(10):
  alpha = round(np.random.uniform(low = 0,high = 5), 2)
  beta = round(np.random.uniform(low = 0, high = 2.5), 2)
  AlphaBetaPairs.append((alpha, beta))

Getting the Gamma Distributions



In [ ]:
GammaDistributions = {}

for A, B in AlphaBetaPairs :
  # A <- alpha , B <- beta
  dataForEachPair = []

  # as numpy takes theta which is 1/beta

  for n in SampleSize :
    data = np.random.gamma(shape = A, scale = 1/B, size = n)
    dataForEachPair.append(data)

  GammaDistributions[(A, B)] = dataForEachPair

# MLE

Obtain the log-likelihood of the gamma function. HINT: You can use the
stats.gamma.logpdf function

Minimize the negative log-likelihood using the scipy minimize function

$$
\text{The likelihood function for the Gamma distribution with rate parameter } \beta \text{ is:}
$$

$$
L(\alpha, \beta \mid x_1, x_2, \dots, x_n) = \prod_{i=1}^{n} \frac{\beta^\alpha}{\Gamma(\alpha)} x_i^{\alpha - 1} e^{-\beta x_i}
$$

$$
\text{The log-likelihood function is:}
$$

$$
\ell(\alpha, \beta) = n \alpha \log \beta - n \log \Gamma(\alpha) + (\alpha - 1) \sum_{i=1}^{n} \log x_i - \beta \sum_{i=1}^{n} x_i
$$


$$
\text{Taking the derivatives of the log-likelihood function with respect to } \alpha \text{ and } \beta, \text{ we obtain:}
$$

### For \( \beta \):

$$
\frac{\partial \ell}{\partial \beta} = \frac{n \alpha}{\beta} - \sum_{i=1}^{n} x_i = 0
$$

Solving for \( \hat{\beta} \):

$$
\hat{\beta} = \frac{n \alpha}{\sum_{i=1}^{n} x_i} = \frac{\alpha}{\bar{x}}
$$

---

### For \( \alpha \):
we use the digamma function \( \psi(\alpha) \):

$$
\frac{\partial \ell}{\partial \alpha} = n \log \beta - n \psi(\alpha) + \sum_{i=1}^{n} \log x_i = 0
$$

Solving for \( \alpha \):

$$
\log \hat{\beta} - \psi(\hat{\alpha}) + \frac{1}{n} \sum_{i=1}^{n} \log x_i = 0
$$

In [ ]:
# getting the negative log likelihood
def negative_log_likelihood(AlphaBetaPairs) :
  alpha, beta = AlphaBetaPairs
  return -np.sum(gamma.logpdf(data, alpha, (1 / beta)))

In [ ]:
# minimizing the negative = getting the maximum
# L-BFGS-B is a good algorithm for dealing with bounds
# the bounds are used to keep the parameters positive
result = minimize(
    negative_log_likelihood,
    [1.0, 1.0],
    method='L-BFGS-B',
    bounds=[(1e-6, None), (1e-6, None)]
)

In [ ]:
# getting the MLE
MLE_alpha, MLE_beta = result.x

# MME

Knowing that E[X] = αβ and Var(X) = αβ
2
. Find the moment estimators for α and β.

### **Method of Moments Estimators (MME)**

For the method of moments, we use the fact that the **mean** and **variance** of the Gamma distribution are:

$$
\mu = \frac{\alpha}{\beta}, \quad \sigma^2 = \frac{\alpha}{\beta^2}
$$

Using sample moments, we estimate the mean and variance:

$$
\hat{\mu} = \frac{1}{n} \sum_{i=1}^{n} x_i, \quad \hat{\sigma}^2 = \frac{1}{n} \sum_{i=1}^{n} (x_i - \hat{\mu})^2
$$

Now, solving for \( \alpha \) and \( \beta \):

### For \( \beta \):

$$
\hat{\beta} = \frac{\hat{\alpha}}{\hat{\mu}}
$$

### For \( \alpha \):

$$
\hat{\alpha} = \frac{\hat{\mu}^2}{\hat{\sigma}^2}
$$

# Plotting